In [1]:
# type:ignore
import time
import sys
import io
import pandas as pd

from gurobipy import *
from functions2 import *
from openpyxl import load_workbook

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# parâmetros
instances = [
            #  'instancia1',
            #  'instancia2',
            #  'instancia3',
            #  'instancia4',
            #  'instancia5',
            #  'instancia6',
            #  'instancia7',
            #  'instancia8',
            #  'instancia9',
            #  'instancia10'
             'test'
            ]

# Qs = [561]#,637,563,561,561,565,491,493,563,493] estos eram los máximos
Qs = {
      # 'instancia1':{'P':561, 'Z':561},
    #   'instancia2':{'P':633, 'Z':633},
      # 'instancia3':{'P':419, 'Z':419},
      # 'instancia4':{'P':561, 'Z':561},
      # 'instancia5':{'P':417, 'Z':417},
      # 'instancia6':{'P':493, 'Z':493},
      # 'instancia7':{'P':491, 'Z':491},
      # 'instancia8':{'P':493, 'Z':493},
      # 'instancia9':{'P':419, 'Z':419},
      # 'instancia10':{'P':493, 'Z':493}
      'test':{'p':561, 'z':561}
      } # estas son las minímas

In [3]:
path_excel = '/home/wilmer/Documentos/Codes/tesis/Instancias/resumo.xlsx'
wb = load_workbook(path_excel)
hoja = wb.active

for instance in instances:
    
    path = '/home/wilmer/Documentos/Codes/tesis/Instancias/'+instance
    Q = Qs[instance]

    modelos = [
               BaseModel#,
            #    BaseModel_Fulfillments,
            #    BaseModel_Skiplagging,
            #    BaseModel_Fulfillments_Skiplagging#,

            #    HierarBehavioralModel,
            #    HierarBehavioralModel_Fulfillments,
            #    HierarBehavioralModel_Skiplagging,
            #    HierarBehavioralModel_Fulfillments_Skiplagging,

            #    PercentBehavioralModel,
            #    PercentBehavioralModel_Fulfillments,
            #    PercentBehavioralModel_Skiplagging,
            #    PercentBehavioralModel_Fulfillments_Skiplagging
            ]

    for classeModelo in modelos:

        # criar modelo
        start_time_cria_model = time.time()

        modelo = classeModelo(
            path_dem = path + '/demanda_e3_t2.csv',
            path_preco = path + '/preco_e3_v2.csv',
            path_rota1 = path + '/rota_e3.csv',
            Q = Q,
            perio = 0
        )

        model, A, X, Y, BY, BX, BL, P, d, perio, indexCombiDem = modelo.create_model()

        end_time_cria_model = time.time()

        # Redigir a saída padrão para um arquivo
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        # Optimizar o modelo
        start_time_opt = time.time()

        model.optimize()
        # model.write(path+'/modelo.lp')
        end_time_opt = time.time()

        # Calcular tempos
        tempoCriacao = end_time_cria_model - start_time_cria_model
        tempoOpt = end_time_opt - start_time_opt

        if model.status == GRB.OPTIMAL:
            # Salvar solução
            a = save_solution(model, BX, BL, P, d, X, Y, A, BY, perio, classeModelo.__name__, indexCombiDem, path)

            print("Modelo: ", classeModelo.__name__)
            print("Instancia: ", instance)
            print("FO: ", model.ObjVal)
            print("FO Relaxada: ", model.ObjBound)
            print("Tempo Criação: ", tempoCriacao)
            print("Tempo da optimização: ", tempoOpt)
        else:
            print("Modelo: ", classeModelo.__name__)
            print("Instancia: ", instance)
            print("FO: ", "Infac")
            print("FO Relaxada: ", model.ObjBound)
            print("Tempo Criação: ", tempoCriacao)
            print("Tempo da optimização: ", tempoOpt)

        # Voltar a saída padrão
        sys.stdout = old_stdout

        # Obter a saída da otimização
        output = new_stdout.getvalue()

        # Salvar a saída num arquivo de texto
        with open( path + '/'+ classeModelo.__name__ +'.txt', 'w') as f:
            f.write(output)

        # Salvar em excel um resumo dos resultados
        if model.status == GRB.OPTIMAL:
            hoja.append([instance, classeModelo.__name__, model.NumVars, tempoCriacao, tempoOpt, model.NodeCount, model.IterCount, model.SolCount, model.MIPGap, model.ObjBound, model.ObjVal])
        else:
            hoja.append([instance, classeModelo.__name__, model.NumVars, tempoCriacao, tempoOpt, "-", "-", "-", "-", "-",'Infac'])
        wb.save(path_excel)



        # Obtener restricciones y variables
        variables = model.getVars()
        restricciones = model.getConstrs()

        # Crear matriz MA
        MA = []
        for constr in restricciones:
            fila = [model.getCoeff(constr, var) for var in variables]  # Obtener coeficiente de cada variable en la restricción
            MA.append(fila)

        # Convertir la matriz A en un DataFrame
        import pandas as pd
        A_df = pd.DataFrame(MA, columns=[var.VarName for var in variables], index=[constr.ConstrName for constr in restricciones])

        # Exportar la matriz A a CSV
        A_df.to_csv(path+"matriz_A.csv", index=True)



        print("Termino a Instancia: " + instance + ", usando o modelo: " + classeModelo.__name__)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-25
Termino a Instancia: test, usando o modelo: BaseModel


In [4]:
import numpy as np
import pandas as pd
from itertools import combinations

def is_unimodular(matrix):
    """
    Comprueba si una matriz es unimodular.
    Una matriz es unimodular si todos sus subdeterminantes son 0, 1 o -1.

    Parámetros:
    matrix (np.array): Matriz cuadrada o rectangular a analizar.

    Retorna:
    True si la matriz es unimodular, False en caso contrario.
    """
    rows, cols = matrix.shape
    min_dim = min(rows, cols)  # Tamaño máximo de las submatrices cuadradas

    for k in range(1, min_dim + 1):  # Orden de submatrices cuadradas
        row_indices = list(combinations(range(rows), k))
        col_indices = list(combinations(range(cols), k))

        for r in row_indices:
            for c in col_indices:
                submatrix = matrix[np.ix_(r, c)]  # Extraer submatriz cuadrada
                det = round(np.linalg.det(submatrix))  # Determinante redondeado
                if det not in [0, 1, -1]:  # Verificar condición de unimodularidad
                    print(f"Submatriz no unimodular encontrada:\n{submatrix}")
                    print(f"Determinante: {det}")
                    return False
    return True

# Cargar la matriz A desde un archivo CSV
file_path = "/home/wilmer/Documentos/Codes/tesis/Instancias/testmatriz_A.csv"  # Asegúrate de reemplazarlo por la ruta correcta
A_df = pd.read_csv(file_path, index_col=0)

# Convertir el DataFrame a un array NumPy
A = A_df.to_numpy()

# Verificar la unimodularidad
if is_unimodular(A):
    print("La matriz A es unimodular.")
else:
    print("La matriz A no es unimodular.")

Submatriz no unimodular encontrada:
[[-561.]]
Determinante: -561
La matriz A no es unimodular.


In [5]:
# attrs = ['Authorizations[Y]','Assignments[X]','Demanda','Preco']

# graf2 = grafica(a,attrs, 2, 8, 4, 2)
# graf1 = grafica(a,attrs, 2, 15, 4, 1)
# graf0 = grafica(a,attrs, 2, 15, 4, 0)

# graf2.savefig(path+'/img2.png', dpi=300, transparent=True)
# graf1.savefig(path+'/img1.png', dpi=300, transparent=True)
# graf0.savefig(path+'/img0.png', dpi=300, transparent=True)


# for instance in instances:

#     path = '/home/wilmer/Documentos/Codes/tesis/Instancias/'+instance
#     data = pd.read_csv(path + '/demanda.csv')
#     data['new'] = data['Vagon'] + data['Class'].astype(str) 
#     # print(instance + " = "+ str(data.DBD.nunique()))
#     print(instance + " = "+ str(data.new.nunique()))
